In [44]:
import sys
sys.path.append('code/transethnic_prs-main/')
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import transethnic_prs.model1.Model1Blk as model1blk
from scipy import optimize
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNetCV

In [47]:
from transethnic_prs.model1.Model1Helper import get_lambda_seq

In [48]:
gene = 'ENSG00000206176.5'
gene_name = 'nan1765'

#def data_processing(gene,gene_name):
    # read eur and afr sample info
eur_sample = pd.read_csv('data/clean/'+gene_name+'_genotype/eur_'+gene_name+'_genotype.012.indv', sep = '\t', header = None)
afr_sample = pd.read_csv('data/clean/'+gene_name+'_genotype/afr_'+gene_name+'_genotype.012.indv', sep = '\t',header = None)
    
    #load 2 genotype matrices
eur_genotype = pd.read_csv('data/clean/'+gene_name+'_genotype/eur_'+gene_name+'_genotype.012', sep = '\t', header = None, index_col = 0)
afr_genotype = pd.read_csv('data/clean/'+gene_name+'_genotype/afr_'+gene_name+'_genotype.012', sep = '\t', header = None, index_col = 0)

    # extract target phenotype vector
pheno_total = pd.read_csv('data/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt', sep = '\t', index_col = 'TargetID')
target_pheno_total = pheno_total.loc[gene]
target_pheno_eur = pd.merge(target_pheno_total, eur_sample, left_index = True, right_on = 0)
target_pheno_afr = pd.merge(target_pheno_total, afr_sample, left_index = True, right_on = 0)
pa = target_pheno_afr.set_index(0)
pe = target_pheno_eur.set_index(0)

    #make sure the sample order of phenotype vector is the same as genotype matrix's
sorted_eur_pheno = pd.merge(eur_sample, pe, left_on = 0 , right_index = True, how = 'left')
sorted_afr_pheno = pd.merge(afr_sample, pa, left_on = 0 , right_index = True, how = 'left')
    
    #return eur_genotype,afr_genotype,sorted_eur_pheno,sorted_afr_pheno

# inpute nan with each SNP's mean value
f = lambda x : x.replace(-1,x.mean())
afr_genotype = afr_genotype.apply(f,axis = 'index')

In [49]:
#original matrix(before standardization)
X2o = np.array(afr_genotype,dtype = np.float64,order = 'C')
y2o = np.array(sorted_afr_pheno[gene],dtype = np.float64,order = 'C')

In [50]:
def check_nan(x):
    count = 0
    for x in x.describe().loc['min']:
        if x == -1:
            count+=1
    return count

In [51]:
def SNP_var_check(X):
    col_valid = []
    count = 0
    for col,x in enumerate(np.std(X, axis = 0)):
        if x==0:
            count +=1
        else:
            col_valid.append(col)
    return col_valid, count

In [52]:
def standardization(x):
    x=np.array(x,dtype = np.float64,order = 'C')
    x_center = x - np.mean(x,axis = 0)
    return x_center/np.std(x, axis = 0)

In [53]:
col_valid1,count1=SNP_var_check(X2o)

In [54]:
X_train,X_test,y_train,y_test = train_test_split(X2o,y2o,test_size = 0.2, random_state = 9,shuffle = False)

In [55]:
col_valid2,count2 = SNP_var_check(X_test)

In [56]:
kf = KFold(n_splits=5)#without shuffling, the random state is immutable

In [57]:
i = 1
for train_index, test_index in kf.split(X_train):
    train_index = list(train_index)
    test_index = list(test_index)
    X_to,X_vo = X_train[train_index,:],X_train[test_index,:]
    y_to, y_vo = y_train[train_index],y_train[test_index]
    col_valid_train,count_train = SNP_var_check(X_to)
    col_valid_valid,count_valid = SNP_var_check(X_vo)
    print("count_train%s:" % i,count_train,"count_validation%s:" % i,count_valid)
    col_valid3 = list(set(col_valid_train).intersection(set(col_valid_valid)))
    globals()['col_set%s' % i] = col_valid3
    i+=1

valid_col_trainset = list(set(col_set1).intersection(col_set2).intersection(col_set3).intersection(col_set4).intersection(col_set5))

count_train1: 1 count_validation1: 3
count_train2: 0 count_validation2: 13
count_train3: 0 count_validation3: 5
count_train4: 0 count_validation4: 12
count_train5: 0 count_validation5: 14


In [58]:
test = list(set(col_valid2).intersection(valid_col_trainset))
len(test)

31

In [59]:
col_valid_final = list(set(col_valid2).intersection(valid_col_trainset))
len(col_valid_final)

31

In [60]:
X_test = X_test[:,col_valid_final]
X_test.shape

(18, 31)

In [61]:
X_train = X_train[:,col_valid_final]

In [62]:
X_test_std = standardization(X_test)
y_test_std = standardization(y_test)

In [63]:
from sklearn.linear_model import ElasticNetCV

In [78]:
lambda_max = 2 * np.absolute(X_train.T @ y_train).max()/0.1
lambda_seq = get_lambda_seq(lambda_max, 100, 100)

In [79]:
lambda_seq

array([2745.399399  , 2620.61675923, 2501.50568303, 2387.80838906,
       2279.27881259, 2175.68207286, 2076.79396483, 1982.40047393,
       1892.29731288, 1806.28947955, 1724.19083499, 1645.82370053,
       1571.01847328, 1499.61325906, 1431.45352202, 1366.39175023,
       1304.28713638, 1245.00527307, 1188.41786195, 1134.40243599,
       1082.84209451, 1033.62525012,  986.64538727,  941.80083169,
        898.99453037,  858.13384152,  819.13033403,  781.89959615,
        746.36105277,  712.43779104,  680.0563939 ,  649.14678124,
        619.64205818,  591.47837032,  564.59476554,  538.93306208,
        514.43772265,  491.05573421,  468.73649322,  447.43169619,
        427.09523506,  407.68309747,  389.1532715 ,  371.46565471,
        354.58196741,  338.46566975,  323.08188269,  308.3973125 ,
        294.38017869,  281.00014524,  268.22825496,  256.0368668 ,
        244.39959606,  233.29125722,  222.68780954,  212.56630491,
        202.90483829,  193.68250024,  184.87933169,  176.47628

In [84]:
model = ElasticNetCV(l1_ratio=0.1, alphas=lambda_seq, cv=kf, n_jobs=-1)
# fit model
model.fit(X_train, y_train)

ElasticNetCV(alphas=array([2745.399399  , 2620.61675923, 2501.50568303, 2387.80838906,
       2279.27881259, 2175.68207286, 2076.79396483, 1982.40047393,
       1892.29731288, 1806.28947955, 1724.19083499, 1645.82370053,
       1571.01847328, 1499.61325906, 1431.45352202, 1366.39175023,
       1304.28713638, 1245.00527307, 1188.41786195, 1134.40243599,
       1082.84209451, 1033.62525012,  986.64538727,  941....
         80.02984927,   76.3923691 ,   72.92021803,   69.60588158,
         66.44218684,   63.4222869 ,   60.53964608,   57.78802573,
         55.16147078,   52.6542968 ,   50.26107774,   47.97663419,
         45.79602212,   43.71452224,   41.72762974,   39.83104457,
         38.02066212,   36.29256435,   34.64301128,   33.06843296,
         31.56542164,   30.13072451,   28.76123658,   27.45399399]),
             cv=KFold(n_splits=5, random_state=None, shuffle=False),
             l1_ratio=0.1, n_jobs=-1)

In [81]:
y_hat_test = model.predict(X_test_std)
model.alpha_

27.453993989959628

In [85]:
y_hat_test

array([1.52876314, 1.4602396 , 1.53105959, 1.49265712, 1.5040559 ,
       1.46467342, 1.52075252, 1.52572974, 1.46262435, 1.51476972,
       1.46543607, 1.52679399, 1.53001595, 1.46372599, 1.46857793,
       1.4568673 , 1.4462974 , 1.52421304])

In [24]:
y_hat_test = (y_hat_test-y_hat_test.mean())/y_hat_test.std()
correlation_test_matrix = np.corrcoef(y_hat_test, y_test_std)
correlation_test = correlation_test_matrix[0,1]
r2 = correlation_test**2


m, b = np.polyfit(y_hat_test,y_test_std, 1)
print(y_hat_test.std())
print(correlation_test,r2,m)
y_test_std.std()

0.0
nan nan 6.521400841331846e-17


/home/daisy/anaconda3/envs/transethnic_prs/lib/python3.7/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/daisy/anaconda3/envs/transethnic_prs/lib/python3.7/site-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/daisy/anaconda3/envs/transethnic_prs/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


1.0

In [30]:
Max1 = 0
for train_index, test_index in kf.split(X_train):
    X_to, X_vo = X_train[:,col_valid_final][train_index,:], X_train[:,col_valid_final][test_index,:]
    y_to, y_vo = y_train[train_index], y_train[test_index]
    X_t_std, X_v_std, y_t_std, y_v_std = standardization(X_to), standardization(X_vo), standardization(y_to), standardization(y_vo)
    mod = model1blk.Model1Blk([A1],[b1],[X_t_std],y_t_std)
    beta_mat_en, lambda_seq_en, niters_en, tols_en, convs_en = mod.solve_path(alpha=0.1)
    print(lambda_seq_end)
    Max2 = 0
    for i in range(100):
        beta_hat = beta_mat_en[:,i]
        y_v_hat = X_v_std @ beta_hat
        correlation_matrix = np.corrcoef(y_v_hat, y_v_std)
        correlation_yvhat_yv = correlation_matrix[0,1]
        r2 = correlation_yvhat_yv**2
        if r2>Max2:
            Max2=r2
            idx = i
    print(Max2)
    if Max2>Max1:
        Max1=Max2
        lam=lambda_seq_en[idx]
        beta_h=beta_mat_en[:,idx]

print(Max1,idx,lam,beta_h)

IndexError: index 183 is out of bounds for axis 1 with size 183

In [40]:
from scipy.stats import pearsonr
r,_ = pearsonr(y_hat_test,y_test_std)
np.corrcoef(y_hat_test,y_test_std)

array([[1.        , 0.72576101],
       [0.72576101, 1.        ]])

In [41]:
from scipy.stats import spearmanr
sp_r, _ = spearmanr(y_hat_test,y_test_std)
sp_r

0.6119711042311662

In [42]:
A = np.vstack([y_test_std, np.ones(len(y_test_std))]).T
m, c = np.linalg.lstsq(A, y_hat_test, rcond=None)[0]
m

0.7257610122566789

1 3
2 7
3 9
